# Project Critique Aribnb Dataset

## Dataset Description

In [1]:
# load packages
import pickle

from subprocess import run
from hashlib import sha256

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline
plt.style.use('seaborn-whitegrid')

random_state = 26498

# import csv of anonymized data
anon_data = pd.read_csv("anonymized_airbnb.csv")
anon_data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,*****,*****,-1540419183100330906,*****,Brooklyn,Kensington,*****,*****,Private room,157.324234,3.548874,6.920243,2018-10,0.21,6,365
1,*****,*****,-6303625318003254822,*****,Manhattan,Midtown,*****,*****,Entire home/apt,222.991412,1.552826,46.989945,2019-05,0.38,2,355
2,*****,*****,2655665583548708572,*****,Manhattan,Harlem,*****,*****,Private room,154.848218,3.455304,0.698164,NaN,NaN,1,365
3,*****,*****,-2939139632100285860,*****,Brooklyn,Clinton Hill,*****,*****,Entire home/apt,87.730973,-9.733857,270.073743,2019-07,4.64,1,194
4,*****,*****,1918944794498209498,*****,Manhattan,East Harlem,*****,*****,Entire home/apt,75.474851,8.164240,4.983751,2018-11,0.10,1,0


## Initial Thoughts

Just to reiterate I wanted to summarize the anonymization approach that was used. Many attributes that were helpful in identification were suppressed. It is important to note that host_id was hashed, so they still exist, but are unique. Review dates were generalized to months and min nights and review numbers have output pertubation.

My first reaction was that hashing the host_id is counteractive to the k-anonymization done on neighborhoud and last review categories. Having a unique host_id still makes each row identifiable by host, even if that host is not know.

## Deanonymization

### Linking Attack


### Homogeneity Attack

## Potential Disclosure

## Overall Rating